In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import numpy as np
from config import CORPUSE_DATA_FILEPATH

In [58]:
# read it in to inspect it
with open(CORPUSE_DATA_FILEPATH, 'r', encoding='utf-8') as f:
    text = f.read()

In [59]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  394290


In [60]:
# let's look at the first 1000 characters
print(text[:1000])


 
 

 
 
 
 
 
 
 
 
 

 1. Óbito do   autor 
 A lgum tempo hesitei se devia abrir estas memórias pelo princípio ou pelo fim, isto é, se poria em primeiro lugar o meu nascimento ou a minha morte. Suposto o uso vulgar seja começar pelo nascimento, duas considerações me levaram a adotar diferente método: a primeira é que eu não sou propriamente um autor defunto, mas um defunto autor, para quem a campa foi outro berço; a segunda é que  o escrito ficaria assim mais galante e mais novo. Moisés, que também contou a sua morte, não a pôs no intróito, mas no cabo: diferença radical entre este livro e o Pentateuco. 
 Dito isto, expirei às duas horas da tarde de uma sexta-feira do mês de agosto de 1869, na minha bela chácara de Catumbi. Tinha uns sessenta e quatro anos, rijos e prósperos, era solteiro, possuía cerca de trezentos contos e fui acompanhado ao cemitério por onze  amigos. Onze amigos! Verdade é que não houve cartas nem anúncios. Acresce que chovia  –  peneirava uma chuvinha miúda, tr

In [61]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !&()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXZ[]_abcdefghijklmnopqrstuvwxyz°´ÀÁÂÃÇÉÍÓÚàáâãçèéêíñóôõúûü–—‘’“”…
113


In [62]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("pelo fim"))
print(decode(encode("pelo fim")))

[68, 57, 64, 67, 2, 58, 61, 65]
pelo fim


In [63]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([394290]) torch.int64
tensor([  1,   2,   1,   2,   1,   1,   2,   1,   2,   1,   2,   1,   2,   1,
          2,   1,   2,   1,   2,   1,   2,   1,   2,   1,   1,   2,  13,  10,
          2,  88,  54,  61,  72,  67,   2,  56,  67,   2,   2,   2,  53,  73,
         72,  67,  70,   2,   1,   2,  25,   2,  64,  59,  73,  65,   2,  72,
         57,  65,  68,  67,   2,  60,  57,  71,  61,  72,  57,  61,   2,  71,
         57,   2,  56,  57,  74,  61,  53,   2,  53,  54,  70,  61,  70,   2,
         57,  71,  72,  53,  71,   2,  65,  57,  65, 100,  70,  61,  53,  71,
          2,  68,  57,  64,  67,   2,  68,  70,  61,  66,  55,  98,  68,  61,
         67,   2,  67,  73,   2,  68,  57,  64,  67,   2,  58,  61,  65,   8,
          2,  61,  71,  72,  67,   2,  96,   8,   2,  71,  57,   2,  68,  67,
         70,  61,  53,   2,  57,  65,   2,  68,  70,  61,  65,  57,  61,  70,
         67,   2,  64,  73,  59,  53,  70,   2,  67,   2,  65,  57,  73,   2,
         66,  53,  71,  55,  61

In [64]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [65]:
block_size = 8
train_data[:block_size+1]

tensor([1, 2, 1, 2, 1, 1, 2, 1, 2])

In [66]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 2
when input is tensor([1, 2]) the target: 1
when input is tensor([1, 2, 1]) the target: 2
when input is tensor([1, 2, 1, 2]) the target: 1
when input is tensor([1, 2, 1, 2, 1]) the target: 1
when input is tensor([1, 2, 1, 2, 1, 1]) the target: 2
when input is tensor([1, 2, 1, 2, 1, 1, 2]) the target: 1
when input is tensor([1, 2, 1, 2, 1, 1, 2, 1]) the target: 2


In [67]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[67,  2, 71, 57, 70, 61, 53,  2],
        [57, 65, 68, 67, 71,  2, 56, 57],
        [97, 66, 55, 61, 53, 71, 23,  2],
        [56, 57,  2, 65, 73, 64, 60, 57]])
targets:
torch.Size([4, 8])
tensor([[ 2, 71, 57, 70, 61, 53,  2, 65],
        [65, 68, 67, 71,  2, 56, 57, 68],
        [66, 55, 61, 53, 71, 23,  2, 72],
        [57,  2, 65, 73, 64, 60, 57, 70]])
----
when input is [67] the target: 2
when input is [67, 2] the target: 71
when input is [67, 2, 71] the target: 57
when input is [67, 2, 71, 57] the target: 70
when input is [67, 2, 71, 57, 70] the target: 61
when input is [67, 2, 71, 57, 70, 61] the target: 53
when input is [67, 2, 71, 57, 70, 61, 53] the target: 2
when input is [67, 2, 71, 57, 70, 61, 53, 2] the target: 65
when input is [57] the target: 65
when input is [57, 65] the target: 68
when input is [57, 65, 68] the target: 67
when input is [57, 65, 68, 67] the target: 71
when input is [57, 65, 68, 67, 71] the target: 2
when input is [57, 

In [68]:
print(xb) # our input to the transformer

tensor([[67,  2, 71, 57, 70, 61, 53,  2],
        [57, 65, 68, 67, 71,  2, 56, 57],
        [97, 66, 55, 61, 53, 71, 23,  2],
        [56, 57,  2, 65, 73, 64, 60, 57]])


In [69]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 113])
tensor(5.0475, grad_fn=<NllLossBackward0>)
	5—´etsÃsTZqCÍ4j(ZI[”)bcoFÇnEXÚ…&â-)5Úd. ]/;ç3B86ÁÂáfCg/L°qCÍ6ÀÀ5d_…CûU3õôm*lâExJüQ1Rdo.aKDCÁTyaJ-— Â


In [70]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [71]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.075191020965576


In [72]:
np.log(1/65)

np.float64(-4.174387269895637)

In [73]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

	tMkPvr3	bÚ4 êóaEhR*w&&(FÃ´üen(ôüK0ãcu88-)nz01°7m…Eúzi	aB 7SfJ…Sv”s1ÇíALK´GaQLêá´5D3Ã)6m_-h;9E6Lh)[ndeIÍ Kú:Z&Ú&l; kIêAPsà°Á_ÁkêÁ”AjÓ*0Q:]uãLT‘1júNmé3W6…,zütQó8[üX*ÓM?Áúe)6U3HÓídPèp9ELo—´SÇ7ÍôóaB?”(–G°A/Câ2’J)7QÇdÂEKUHJÓ2)D]Á_ipR	aHõuú8PZ*ÍûbTWT”(g(y).S6É6EçV!etUõôyB;Ojé&…í Í;b’Q:]ÃZ*7v”V
Áce°ÁnçVS°ü*c;MÀz–n_hévcüÃd!M)Upp9rg,t,1ãN´_pkd—´EIFoU3õÂâ—9NjP6-PIñXñá´	/óhûgpyf0b5-ctóÓx*7pPvaõNÇÚUIH–Nrhqú5ÃjÚlmÍàIeBEM…:qx”lK-Í,ãnQkÓ5byêíá-ÉP067rh‘ô–VúUP
x‘9ñô?aûzüP?qM!]íá?“ñ2]gã,W6‘5Oy96Ã—Ó2ç!0uaûázjLU[!	


In [74]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [75]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [76]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [77]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [78]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [79]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [80]:
torch.manual_seed(42)
#a = torch.ones(3,3)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b 
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [81]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [82]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [89]:
torch.manual_seed(42)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [90]:
wei[0]

tensor([[0.0839, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0606, 0.2418, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1684, 0.0240, 0.2433, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.2614, 0.1076, 0.3452, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1187, 0.0218, 0.0152, 0.0890, 0.0695, 0.0000, 0.0000, 0.0000],
        [0.0654, 0.3663, 0.0801, 0.3100, 0.6478, 0.0812, 0.0000, 0.0000],
        [0.3644, 0.0150, 0.5266, 0.0205, 0.1464, 0.0097, 0.0605, 0.0000],
        [0.0326, 0.0698, 0.0271, 0.2353, 0.1363, 0.9091, 0.9395, 1.0000]],
       grad_fn=<SelectBackward0>)

In [85]:
x

tensor([[[ 1.9269,  1.4873,  0.9007,  ...,  0.0418, -0.2516,  0.8599],
         [-1.3847, -0.8712, -0.2234,  ...,  1.8446, -1.1845,  1.3835],
         [ 1.4451,  0.8564,  2.2181,  ..., -0.8278,  1.3347,  0.4835],
         ...,
         [-1.9006,  0.2286,  0.0249,  ..., -0.5558,  0.7043,  0.7099],
         [ 1.7744, -0.9216,  0.9624,  ..., -0.5003,  1.0350,  1.6896],
         [-0.0045,  1.6668,  0.1539,  ...,  0.5655,  0.5058,  0.2225]],

        [[-0.6855,  0.5636, -1.5072,  ...,  1.1566,  0.2691, -0.0366],
         [ 0.9733, -1.0151, -0.5419,  ..., -0.0553,  1.2049, -0.9825],
         [ 0.4334, -0.7172,  1.0554,  ..., -0.6766, -0.5730, -0.3303],
         ...,
         [ 0.6839, -1.3246, -0.5161,  ...,  1.1895,  0.7607, -0.7463],
         [-1.3839,  0.4869, -1.0020,  ...,  1.9535,  2.0487, -1.0880],
         [ 1.6217,  0.8513, -0.4005,  ...,  0.4232, -0.3389,  0.5180]],

        [[-1.3638,  0.1930, -0.6103,  ...,  0.6110,  1.2208, -0.6076],
         [-1.7376, -0.1254, -1.3658,  ..., -0